In [1]:
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
from mindnlp.dataset import load_dataset

import mindspore
from mindspore import ops
import os

from mindnlp.transformers import AutoTokenizer
from mindnlp.modules.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_lora_v1.ckpt"
max_length = 128
lr = 1e-3
num_epochs = 3
batch_size = 8

/home/daiyuxin/miniconda3/envs/ms2.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.594 seconds.
Prefix dict has been built successfully.


In [2]:
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.19151053100118282


In [4]:
mindspore.dataset.config.set_seed(123)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

/home/daiyuxin/.local/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [6]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:01:39.237.921 [mindspore/dataset/engine/datasets.py:1203] Dataset is shuffled before split.


In [7]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [8]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= 'The gross area of the Innova 2 project will be about 10,000 sq m ( 107,600 sq ft ) .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [10]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [11]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[   486,  33720,   9340 ...      0,      0,      0],
  [   259,  85336,    714 ...      0,      0,      0],
  [   486,  21511,    345 ...      0,      0,      0],
  ...
  [169774,    262,    639 ...      0,      0,      0],
  [  9981,    259,  45814 ...      0,      0,      0],
  [   486,  21982,    345 ...      0,      0,      0]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 3], dtype=Int64, value=
 [[59006,     1,  -100],
  [59006,     1,  -100],
  [59006,     1,  -100],
  ...
  [59006,     1,  -100],
  [18205,     1,  -100],
  [59006,     1,  -100]])}

In [12]:
# optimizer and lr scheduler
optimizer = mindspore.experimental.optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [13]:
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:08<00:00,  3.24it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.37888) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.321273) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05301) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0516489)


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:08<00:00,  3.26it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.06363) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0616836) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.03097) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0304957)


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:08<00:00,  3.25it/s]

epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.04172) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0408725) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.04111) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0402883)


In [14]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=96.01769911504425 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'positive', 'positive']
ground_truth[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive']


In [15]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[   486,   2733,   6396 ...      0,      0,      0],
  [   486,    259, 150106 ...      0,      0,      0],
  [   486,   5835,    259 ...      0,      0,      0],
  ...
  [  1385,    339,    259 ...      0,      0,      0],
  [   259,  18948,    776 ...      0,      0,      0],
  [   486,    259,  20147 ...      0,      0,      0]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 3], dtype=Int64, value=
 [[59006,     1,  -100],
  [59006,     1,  -100],
  [59006,     1,  -100],
  ...
  [59006,     1,  -100],
  [59006,     1,  -100],
  [18205,     1,  -100]])]

In [16]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [17]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


9.1M	bigscience/mt0-large_LORA_SEQ_2_SEQ_LM/adapter_model.ckpt


In [18]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:19:04.209.624 [mindspore/train/serialization.py:1378] For 'load_param_into_net', 558 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:19:04.210.646 [mindspore/train/serialization.py:1383] base_model.shared.weight is not loaded.
[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:19:04.211.076 [mindspore/train/serialization.py:1383] base_model.encoder.block.0.layer.0.SelfAttention.q.weight is not loaded.
[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:19:04.211.434 [mindspore/train/serialization.py:1383] base_model.encoder.block.0.layer.0.SelfAttention.k.weight is not loaded.
[WARNING] ME(19505:139997890737984,MainProcess):2024-05-23-10:19:04.211.769 [mindspore/train/serialization.py:1383] base_model.encoder.blo

In [22]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))

print(example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")
print(inputs)

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

neutral
{'input_ids': Tensor(shape=[1, 2], dtype=Int64, value=
[[59006,     1]]), 'attention_mask': Tensor(shape=[1, 2], dtype=Int64, value=
[[1, 1]])}


/home/daiyuxin/miniconda3/envs/ms2.0/lib/python3.8/site-packages/mindnlp/transformers/generation/utils.py:1402: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[[    0 59006     1]]
['neutral']
